# Label classifier (3dshapes): data collection

## Task 2

**Author**: Maleakhi A. Wijaya  
**Description**: This notebook contains code used to collect experimentation data. We compare the performance of methods discussed in Rabanset et al. against our proposed CBSD method. The end-to-end task for task 2 is to predict the combination of scale and shape.

In [1]:
# Load utilities functions
%run ../../scripts/constants.py
%run ../../scripts/3dshapes_utils.py
%run ../../scripts/shift_applicator.py
%run ../../scripts/shift_dimensionality_reductor.py
%run ../../scripts/experiment_utils.py
%run ../../scripts/shift_statistical_test.py

In [2]:
## Random seed
SEED = 20
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load dataset

In [3]:
files_dir = "../../data/3dshapes.h5"
# index 0 = image category
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split_3dshapes(files_dir, 70000, DatasetTask.Task2, 
                                                                               train_size=0.80, class_index=[3, 4])

Training samples: 56000
Testing samples: 14000


In [4]:
n_classes = len(np.unique(np.concatenate([y_train, y_test])))
concept_names = SHAPES3D_CONCEPT_NAMES
concept_values = [len(np.unique(np.concatenate([c_train[:, i], c_test[:, i]]))) for i in range(c_train.shape[1])]

# Split training into validation set as well 
X_train, X_valid = X_train[:40000], X_train[40000:]
y_train, y_valid = y_train[:40000], y_train[40000:]
c_train, c_valid = c_train[:40000], c_train[40000:]

In [5]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = X_test.shape[1:] # constant hold the image original shape
X_test_flatten = deepcopy(X_test.reshape(X_test.shape[0], -1))
X_train_flatten = deepcopy(X_train.reshape(X_train.shape[0], -1))
X_valid_flatten = deepcopy(X_valid.reshape(X_valid.shape[0], -1))

## Dimensionality reduction

We implemented various dimensionality reduction methods, amounting to:
- End to end model (label classifiers/ BBSD)
- Concept bottleneck model (CBSD)
- Principal component analysis (PCA)
- Sparse random projection (SRP)

### End-to-end model

In [6]:
path = "../../models/end_to_end_3dshapes_task2"
# For training and saving
histories, end_to_end_model = end_to_end_neural_network(n_classes, Dataset.SHAPES3D, 
                         X_train, y_train, X_valid, y_valid, path)

# For loading
end_to_end_model = tf.keras.models.load_model(path)

Epoch 1/200
625/625 [==============================] - 11s 18ms/step - loss: 3.4466 - accuracy: 0.0462 - val_loss: 3.1825 - val_accuracy: 0.0845
Epoch 2/200
625/625 [==============================] - 8s 13ms/step - loss: 2.6532 - accuracy: 0.1920 - val_loss: 1.3059 - val_accuracy: 0.7234
Epoch 3/200
625/625 [==============================] - 8s 13ms/step - loss: 1.3000 - accuracy: 0.5482 - val_loss: 0.1587 - val_accuracy: 0.9854
Epoch 4/200
625/625 [==============================] - 9s 14ms/step - loss: 0.5038 - accuracy: 0.8230 - val_loss: 0.0222 - val_accuracy: 0.9994
Epoch 5/200
625/625 [==============================] - 9s 14ms/step - loss: 0.2375 - accuracy: 0.9202 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 6/200
625/625 [==============================] - 8s 13ms/step - loss: 0.1459 - accuracy: 0.9506 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 7/200
625/625 [==============================] - 8s 13ms/step - loss: 0.1023 - accuracy: 0.9656 - val_loss: 4.6279e-04 - val_accu

In [7]:
# Evaluate model
y_pred = end_to_end_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       452
           1       1.00      1.00      1.00       425
           2       1.00      1.00      1.00       420
           3       1.00      1.00      1.00       391
           4       1.00      1.00      1.00       414
           5       1.00      1.00      1.00       428
           6       1.00      1.00      1.00       411
           7       1.00      1.00      1.00       451
           8       1.00      1.00      1.00       424
           9       1.00      1.00      1.00       497
          10       1.00      1.00      1.00       448
          11       1.00      1.00      1.00       431
          12       1.00      1.00      1.00       444
          13       1.00      1.00      1.00       433
          14       1.00      1.00      1.00       460
          15       1.00      1.00      1.00       437
          16       1.00      1.00      1.00       441
          17       1.00    

### Concept bottleneck model

**Input to Concept**

In [8]:
path = "../../models/multitask_3dshapes"
# For training and saving
histories, mt_model = multitask_model(Dataset.SHAPES3D,
                                            X_train, c_train,
                                            X_valid, c_valid, path, concept_values)

# For loading
mt_model = tf.keras.models.load_model(path)

In [9]:
# Evaluate model
for i, pred in enumerate(mt_model.predict(X_test)):
    print("*"*20, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*20)
    c_truth = c_test[:, i]
    c_pred = np.argmax(pred, axis=1)
    
    print(classification_report(c_truth, c_pred))
    print("\n\n")

******************** Model: category ********************
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2426
           1       1.00      1.00      1.00      2439
           2       1.00      1.00      1.00      2429
           3       1.00      1.00      1.00      2426
           4       1.00      1.00      1.00      2430

    accuracy                           1.00     12150
   macro avg       1.00      1.00      1.00     12150
weighted avg       1.00      1.00      1.00     12150




******************** Model: instance ********************
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      1263
           1       0.90      0.86      0.88      1209
           2       0.90      0.91      0.91      1167
           3       0.89      0.87      0.88      1197
           4       0.90      0.89      0.89      1210
           5       0.92      0.87      0.89      1261
           6 

**Concept to Output**

In [10]:
# Build and train model. For simplicity, we used logistic regression
# although can be substituted using other model.
com = LogisticRegression()
com.fit(c_train, y_train)

LogisticRegression()

In [11]:
y_test_pred = com.predict(c_test)
print(classification_report(y_test_pred, y_test))
print(confusion_matrix(y_test_pred, y_test))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       372
           1       0.75      0.95      0.84       327
           2       0.96      0.79      0.87       499
           3       0.80      0.89      0.84       345
           4       0.84      0.82      0.83       414
           5       1.00      0.94      0.97       454
           6       0.82      0.89      0.85       364
           7       0.77      0.90      0.83       356
           8       0.94      0.97      0.96       433
           9       1.00      1.00      1.00       394
          10       1.00      1.00      1.00       390
          11       1.00      1.00      1.00       392
          12       0.94      0.84      0.89       449
          13       0.98      1.00      0.99       383
          14       0.88      0.77      0.83       455
          15       0.99      0.99      0.99       391
          16       0.97      0.95      0.96       443
          17       1.00    

### Principal component analysis

In [12]:
pca, n_components = principal_components_analysis(X_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

The number of components to explain 80% of variance is 2.


### Sparse random projection

In [13]:
srp, n_components = sparse_random_projection(X_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

The number of components to explain 80% of variance is 2.


## Data collection

This section performs various experiments to collect data. We consider various dimensionality reduced methods discussed in the paper and thesis.

### PCA

In [6]:
method = DimensionalityReductor.PCA
model = pca
method_str = "PCA"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task2"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

Saving successfully.


### SRP

In [36]:
method = DimensionalityReductor.SRP
model = srp
method_str = "SRP"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task2"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

Saving successfully.


### BBSDs

In [36]:
method = DimensionalityReductor.BBSDs
model = end_to_end_model
method_str = "BBSDs"

#### Knockout shift

In [6]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task2"

In [6]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [27]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

Saving successfully.


#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor_task2",
    "wall_task2",
    "object_scale_task2",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts

In [ ]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation_task2",
    "shear_task2",
    "flip_task2",
    "all_task2"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian_task2"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### BBSDh

In [36]:
method = DimensionalityReductor.BBSDh
model = end_to_end_model
method_str = "BBSDh"

#### Knockout shift

In [6]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task2"

In [6]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [27]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

Saving successfully.


#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor_task2",
    "wall_task2",
    "object_scale_task2",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts

In [ ]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation_task2",
    "shear_task2",
    "flip_task2",
    "all_task2"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian_task2"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### CBSDs

In [36]:
method = DimensionalityReductor.CBSDs
model = mt_model
method_str = "CBSDs"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task2"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [9]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

Saving successfully.


### CBSDh

In [36]:
method = DimensionalityReductor.CBSDh
model = mt_model
method_str = "CBSDh"

#### Knockout shift

In [7]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko_task2"

In [8]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [1]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")